In [ ]:
# example of training a cyclegan on the horse2zebra dataset
import tensorflow as tf
from random import random
from numpy import load
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from matplotlib import pyplot


# define the discriminator model
def define_discriminator(image_shape):
# weight initialization
 init = RandomNormal(stddev=0.02)
# source image input
 in_image = Input(shape=image_shape)
# C64
 d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
 d = LeakyReLU(alpha=0.2)(d)
	# C128
	d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C256
	d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# C512
	d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
	d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
	d = InstanceNormalization(axis=-1)(d)
	d = LeakyReLU(alpha=0.2)(d)
	# patch output
	patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
	# define model
	model = Model(in_image, patch_out)
	# compile model
	model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0.5), loss_weights=[0.5])
	return model

# generator a resnet block
def resnet_block(n_filters, input_layer):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# first layer convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(input_layer)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# second convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	# concatenate merge channel-wise with input layer
	g = Concatenate()([g, input_layer])
	return g

# define the standalone generator model
def define_generator(image_shape, n_resnet=9):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape)
	# c7s1-64
	g = Conv2D(64, (7,7), padding='same', kernel_initializer=init)(in_image)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d128
	g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d256
	g = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# R256
	for _ in range(n_resnet):
		g = resnet_block(256, g)
	# u128
	g = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# u64
	g = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# c7s1-3
	g = Conv2D(3, (7,7), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	out_image = Activation('tanh')(g)
	# define model
	model = Model(in_image, out_image)
	return model

# define a composite model for updating generators by adversarial and cycle loss
def define_composite_model(g_model_1, d_model, g_model_2, image_shape):
	# ensure the model we're updating is trainable
	g_model_1.trainable = True
	# mark discriminator as not trainable
	d_model.trainable = False
	# mark other generator model as not trainable
	g_model_2.trainable = False
	# discriminator element
	input_gen = Input(shape=image_shape)
	gen1_out = g_model_1(input_gen)
	output_d = d_model(gen1_out)
	# identity element
	input_id = Input(shape=image_shape)
	output_id = g_model_1(input_id)
	# forward cycle
	output_f = g_model_2(gen1_out)
	# backward cycle
	gen2_out = g_model_2(input_id)
	output_b = g_model_1(gen2_out)
	# define model graph
	model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
	# define optimization algorithm configuration
	opt = Adam(lr=0.0002, beta_1=0.5)
	# compile model with weighting of least squares loss and L1 loss
	model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer=opt)
	return model

# load and prepare training images
def load_real_samples(filename):
	# load the dataset
	data = load(filename)
	# unpack arrays
	X1, X2 = data['arr_0'], data['arr_1']
	# scale from [0,255] to [-1,1]
	X1 = (X1 - 127.5) / 127.5
	X2 = (X2 - 127.5) / 127.5
	return [X1, X2]

# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, patch_shape, patch_shape, 1))
	return X, y

# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, dataset, patch_shape):
	# generate fake instance
	X = g_model.predict(dataset)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

# save the generator models to file
def save_models(step, g_model_AtoB, g_model_BtoA):
	# save the first generator model
	filename1 = 'g_model_AtoB_%06d.h5' % (step+1)
	g_model_AtoB.save(filename1)
	# save the second generator model
	filename2 = 'g_model_BtoA_%06d.h5' % (step+1)
	g_model_BtoA.save(filename2)
	print('>Saved: %s and %s' % (filename1, filename2))

# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, trainX, name, n_samples=5):
	# select a sample of input images
	X_in, _ = generate_real_samples(trainX, n_samples, 0)
	# generate translated images
	X_out, _ = generate_fake_samples(g_model, X_in, 0)
	# scale all pixels from [-1,1] to [0,1]
	X_in = (X_in + 1) / 2.0
	X_out = (X_out + 1) / 2.0
	# plot real images
	for i in range(n_samples):
		pyplot.subplot(2, n_samples, 1 + i)
		pyplot.axis('off')
		pyplot.imshow(X_in[i])
	# plot translated image
	for i in range(n_samples):
		pyplot.subplot(2, n_samples, 1 + n_samples + i)
		pyplot.axis('off')
		pyplot.imshow(X_out[i])
	# save plot to file
	filename1 = '%s_generated_plot_%06d.png' % (name, (step+1))
	pyplot.savefig(filename1)
	pyplot.close()

# update image pool for fake images
def update_image_pool(pool, images, max_size=50):
	selected = list()
	for image in images:
		if len(pool) < max_size:
			# stock the pool
			pool.append(image)
			selected.append(image)
		elif random() < 0.5:
			# use image, but don't add it to the pool
			selected.append(image)
		else:
			# replace an existing image and use replaced image
			ix = randint(0, len(pool))
			selected.append(pool[ix])
			pool[ix] = image
	return asarray(selected)
# train cyclegan models
def train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset):
	# define properties of the training run
	n_epochs, n_batch, = 30, 1
	# determine the output square shape of the discriminator
	n_patch = d_model_A.output_shape[1]
	# unpack dataset
	trainA, trainB = dataset
	# prepare image pool for fakes
	poolA, poolB = list(), list()
	# calculate the number of batches per training epoch
	bat_per_epo = int(len(trainA) / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# manually enumerate epochs
	for i in range(n_steps):
		# select a batch of real samples
		X_realA, y_realA = generate_real_samples(trainA, n_batch, n_patch)
		X_realB, y_realB = generate_real_samples(trainB, n_batch, n_patch)
		# generate a batch of fake samples
		X_fakeA, y_fakeA = generate_fake_samples(g_model_BtoA, X_realB, n_patch)
		X_fakeB, y_fakeB = generate_fake_samples(g_model_AtoB, X_realA, n_patch)
		# update fakes from pool
		X_fakeA = update_image_pool(poolA, X_fakeA)
		X_fakeB = update_image_pool(poolB, X_fakeB)
		# update generator B->A via adversarial and cycle loss
		g_loss2, _, _, _, _  = c_model_BtoA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])
		# update discriminator for A -> [real/fake]
		dA_loss1 = d_model_A.train_on_batch(X_realA, y_realA)
		dA_loss2 = d_model_A.train_on_batch(X_fakeA, y_fakeA)
		# update generator A->B via adversarial and cycle loss
		g_loss1, _, _, _, _ = c_model_AtoB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])
		# update discriminator for B -> [real/fake]
		dB_loss1 = d_model_B.train_on_batch(X_realB, y_realB)
		dB_loss2 = d_model_B.train_on_batch(X_fakeB, y_fakeB)
		# summarize performance
		print('>%d, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+1, dA_loss1,dA_loss2, dB_loss1,dB_loss2, g_loss1,g_loss2))
		# evaluate the model performance every so often
		if (i+1) % (bat_per_epo * 1) == 0:
			# plot A->B translation
			summarize_performance(i, g_model_AtoB, trainA, 'AtoB')
			# plot B->A translation
			summarize_performance(i, g_model_BtoA, trainB, 'BtoA')
		if (i+1) % (bat_per_epo * 5) == 0:
			# save the models
			save_models(i, g_model_AtoB, g_model_BtoA)

# load image data
dataset = load_real_samples('horse2zebra_256.npz')
print('Loaded', dataset[0].shape, dataset[1].shape)
# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]
# generator: A -> B
g_model_AtoB = define_generator(image_shape)
# generator: B -> A
g_model_BtoA = define_generator(image_shape)
# discriminator: A -> [real/fake]
d_model_A = define_discriminator(image_shape)
# discriminator: B -> [real/fake]
d_model_B = define_discriminator(image_shape)
# composite: A -> B -> [real/fake, A]
c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
# composite: B -> A -> [real/fake, B]
c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)
# train models
train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset)



Loaded (1187, 256, 256, 3) (1474, 256, 256, 3)


C:\Users\Bich Ninh\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


>1, dA[1.003,1.152] dB[2.133,3.628] g[21.160,18.302]
>2, dA[5.257,0.872] dB[1.278,3.917] g[18.042,20.598]
>3, dA[5.361,1.082] dB[1.311,3.480] g[22.232,18.704]
>4, dA[1.131,1.515] dB[3.182,1.299] g[20.556,15.694]
>5, dA[1.097,2.316] dB[4.034,1.630] g[19.966,18.565]
>6, dA[1.238,2.116] dB[2.466,1.157] g[16.939,18.032]
>7, dA[1.717,2.591] dB[1.940,1.654] g[17.517,19.737]
>8, dA[2.052,2.137] dB[2.276,4.735] g[18.080,14.971]
>9, dA[2.390,2.336] dB[1.334,3.131] g[14.873,14.803]
>10, dA[4.119,2.232] dB[0.742,2.645] g[13.857,14.616]
>11, dA[3.823,1.485] dB[0.727,2.715] g[13.690,14.533]
>12, dA[1.577,0.961] dB[0.855,1.157] g[13.545,13.345]
>13, dA[0.859,0.743] dB[1.721,0.740] g[14.459,13.705]
>14, dA[0.720,0.483] dB[1.827,0.779] g[13.517,14.402]
>15, dA[0.820,0.616] dB[0.737,0.673] g[14.011,13.582]
>16, dA[0.863,0.624] dB[0.616,0.581] g[11.468,11.415]
>17, dA[0.481,0.522] dB[0.642,0.756] g[12.096,13.189]
>18, dA[0.750,0.615] dB[0.553,0.598] g[9.662,9.195]
>19, dA[0.450,0.495] dB[0.477,0.480] g[

>157, dA[0.167,0.111] dB[0.207,0.152] g[6.886,6.806]
>158, dA[0.111,0.155] dB[0.293,0.242] g[6.091,6.706]
>159, dA[0.113,0.114] dB[0.209,0.154] g[8.030,8.205]
>160, dA[0.169,0.095] dB[0.204,0.179] g[8.726,10.161]
>161, dA[0.215,0.079] dB[0.203,0.175] g[6.884,7.009]
>162, dA[0.120,0.130] dB[0.220,0.225] g[7.055,7.418]
>163, dA[0.122,0.177] dB[0.176,0.241] g[7.097,8.033]
>164, dA[0.276,0.296] dB[0.226,0.233] g[7.784,9.017]
>165, dA[0.186,0.155] dB[0.202,0.171] g[6.882,7.102]
>166, dA[0.300,0.097] dB[0.153,0.148] g[9.006,9.531]
>167, dA[0.261,0.147] dB[0.202,0.206] g[7.344,7.078]
>168, dA[0.152,0.090] dB[0.182,0.194] g[6.067,6.603]
>169, dA[0.137,0.183] dB[0.171,0.104] g[6.219,6.620]
>170, dA[0.114,0.134] dB[0.188,0.144] g[6.882,7.426]
>171, dA[0.066,0.076] dB[0.153,0.139] g[7.171,7.143]
>172, dA[0.237,0.192] dB[0.218,0.188] g[6.754,7.658]
>173, dA[0.171,0.151] dB[0.194,0.197] g[6.762,8.560]
>174, dA[0.083,0.104] dB[0.138,0.136] g[8.696,8.638]
>175, dA[0.137,0.101] dB[0.181,0.073] g[6.268

>312, dA[0.283,0.181] dB[0.237,0.226] g[5.284,5.772]
>313, dA[0.199,0.071] dB[0.092,0.160] g[7.546,8.440]
>314, dA[0.159,0.245] dB[0.143,0.124] g[6.699,6.802]
>315, dA[0.156,0.084] dB[0.341,0.190] g[7.079,7.098]
>316, dA[0.059,0.077] dB[0.277,0.188] g[6.123,5.951]
>317, dA[0.056,0.111] dB[0.138,0.222] g[5.868,7.392]
>318, dA[0.102,0.106] dB[0.072,0.207] g[7.135,7.587]
>319, dA[0.308,0.165] dB[0.110,0.128] g[6.871,6.979]
>320, dA[0.033,0.257] dB[0.133,0.139] g[5.650,5.911]
>321, dA[0.387,0.224] dB[0.249,0.196] g[4.641,5.860]
>322, dA[0.083,0.206] dB[0.127,0.060] g[7.099,6.571]
>323, dA[0.205,0.235] dB[0.291,0.237] g[5.927,6.430]
>324, dA[0.159,0.223] dB[0.157,0.155] g[5.095,5.453]
>325, dA[0.172,0.187] dB[0.224,0.107] g[5.260,5.293]
>326, dA[0.164,0.073] dB[0.101,0.087] g[5.805,6.013]
>327, dA[0.106,0.099] dB[0.109,0.161] g[6.696,7.553]
>328, dA[0.286,0.493] dB[0.236,0.081] g[5.250,5.998]
>329, dA[0.120,0.107] dB[0.183,0.121] g[4.758,4.902]
>330, dA[0.219,0.077] dB[0.222,0.194] g[5.651,

...

>1693, dA[0.197,0.120] dB[0.039,0.017] g[6.586,6.682]
>1694, dA[0.344,0.268] dB[0.136,0.018] g[4.928,4.375]
>1695, dA[0.161,0.112] dB[0.049,0.103] g[5.147,5.343]
>1696, dA[0.163,0.104] dB[0.118,0.048] g[5.680,5.880]
>1697, dA[0.098,0.110] dB[0.025,0.194] g[4.589,4.747]
>1698, dA[0.047,0.065] dB[0.485,0.212] g[4.560,4.643]
>1699, dA[0.038,0.109] dB[0.091,0.094] g[6.338,7.414]
>1700, dA[0.028,0.142] dB[0.069,0.130] g[5.379,5.721]
>1701, dA[0.122,0.041] dB[0.029,0.015] g[5.376,6.170]
>1702, dA[0.265,0.066] dB[0.323,0.144] g[6.806,6.572]
>1703, dA[0.070,0.110] dB[0.226,0.273] g[3.516,3.636]
>1704, dA[0.051,0.454] dB[0.282,0.111] g[5.637,5.977]
>1705, dA[0.314,0.053] dB[0.029,0.094] g[4.880,4.862]
>1706, dA[0.245,0.196] dB[0.186,0.072] g[5.275,5.096]
>1707, dA[0.351,0.102] dB[0.046,0.089] g[5.073,5.166]
>1708, dA[0.067,0.230] dB[0.256,0.092] g[6.117,5.742]
>1709, dA[0.208,0.203] dB[0.154,0.149] g[4.168,4.359]
>1710, dA[0.166,0.583] dB[0.017,0.115] g[4.559,4.648]
>1711, dA[0.112,0.040] dB[0.

>1845, dA[0.016,0.029] dB[0.120,0.198] g[5.260,6.148]
>1846, dA[0.319,0.056] dB[0.153,0.216] g[3.662,3.778]
>1847, dA[0.035,0.112] dB[0.398,0.121] g[5.481,5.440]
>1848, dA[0.071,0.089] dB[0.175,0.141] g[4.575,4.494]
>1849, dA[0.102,0.136] dB[0.070,0.132] g[4.812,5.043]
>1850, dA[0.068,0.045] dB[0.285,0.189] g[4.471,4.197]
>1851, dA[0.061,0.092] dB[0.067,0.089] g[4.765,5.598]
>1852, dA[0.182,0.070] dB[0.014,0.048] g[5.320,5.487]
>1853, dA[0.080,0.073] dB[0.092,0.050] g[5.405,4.803]
>1854, dA[0.047,0.053] dB[0.015,0.120] g[5.171,5.467]
>1855, dA[0.252,0.396] dB[0.279,0.091] g[4.991,4.541]
>1856, dA[0.005,0.057] dB[0.024,0.091] g[6.421,6.255]
>1857, dA[0.043,0.018] dB[0.080,0.011] g[9.661,8.443]
>1858, dA[0.147,0.037] dB[0.145,0.096] g[4.578,4.588]
>1859, dA[0.039,0.110] dB[0.013,0.057] g[5.810,6.255]
>1860, dA[0.029,0.240] dB[0.341,0.212] g[4.477,4.592]
>1861, dA[0.023,0.032] dB[0.259,0.135] g[3.297,4.520]
>1862, dA[0.182,0.052] dB[0.074,0.358] g[5.799,7.144]
>1863, dA[0.007,0.137] dB[0.

>1997, dA[0.241,0.128] dB[0.059,0.192] g[5.207,5.932]
>1998, dA[0.035,0.153] dB[0.299,0.200] g[6.342,7.338]
>1999, dA[0.016,0.024] dB[0.126,0.125] g[8.639,10.122]
>2000, dA[0.259,0.044] dB[0.257,0.033] g[4.252,5.415]
>2001, dA[0.025,0.037] dB[0.122,0.156] g[7.927,7.513]
>2002, dA[0.110,0.081] dB[0.123,0.071] g[6.473,6.178]
>2003, dA[0.064,0.148] dB[0.168,0.039] g[5.777,5.036]
>2004, dA[0.348,0.108] dB[0.088,0.712] g[4.324,5.598]
>2005, dA[0.038,0.226] dB[0.359,0.010] g[5.428,4.874]
>2006, dA[0.208,0.050] dB[0.329,0.154] g[5.015,4.881]
>2007, dA[0.197,0.751] dB[0.027,0.173] g[4.101,4.115]
>2008, dA[0.355,0.136] dB[0.074,0.137] g[4.039,4.891]
>2009, dA[0.257,0.044] dB[0.518,0.046] g[8.165,8.328]
>2010, dA[0.228,0.104] dB[0.055,0.406] g[3.816,4.065]
>2011, dA[0.059,0.093] dB[0.238,0.059] g[4.201,3.829]
>2012, dA[0.083,0.131] dB[0.263,0.192] g[5.676,5.996]
>2013, dA[0.043,0.050] dB[0.102,0.229] g[3.992,4.213]
>2014, dA[0.098,0.103] dB[0.072,0.095] g[5.844,6.767]
>2015, dA[0.041,0.088] dB[0

>2149, dA[0.060,0.104] dB[0.053,0.143] g[4.834,5.282]
>2150, dA[0.008,0.111] dB[0.448,0.036] g[4.641,4.868]
>2151, dA[0.324,0.026] dB[0.080,0.100] g[5.393,5.915]
>2152, dA[0.013,0.064] dB[0.101,0.535] g[4.325,5.298]
>2153, dA[0.383,0.053] dB[0.194,0.041] g[4.499,5.385]
>2154, dA[0.055,0.325] dB[0.246,0.138] g[5.827,5.546]
>2155, dA[0.307,0.091] dB[0.049,0.097] g[4.547,4.423]
>2156, dA[0.127,0.127] dB[0.096,0.244] g[3.801,4.624]
>2157, dA[0.202,0.229] dB[0.352,0.224] g[3.953,3.828]
>2158, dA[0.013,0.060] dB[0.028,0.018] g[3.936,3.341]
>2159, dA[0.192,0.269] dB[0.244,0.174] g[4.275,3.888]
>2160, dA[0.048,0.057] dB[0.065,0.132] g[7.343,9.612]
>2161, dA[0.227,0.059] dB[0.072,0.061] g[3.904,4.488]
>2162, dA[0.217,0.172] dB[0.015,0.228] g[4.624,4.920]
>2163, dA[0.099,0.248] dB[0.630,0.036] g[5.485,4.702]
>2164, dA[0.081,0.125] dB[0.272,0.412] g[3.797,4.032]
>2165, dA[0.029,0.035] dB[0.125,0.102] g[4.560,5.500]
>2166, dA[0.235,0.515] dB[0.054,0.101] g[6.600,6.302]
>2167, dA[0.015,0.034] dB[0.

>2301, dA[0.093,0.015] dB[0.044,0.064] g[5.437,5.440]
>2302, dA[0.103,0.271] dB[0.185,0.050] g[7.429,7.868]
>2303, dA[0.107,0.078] dB[0.045,0.293] g[4.273,4.902]
>2304, dA[0.018,0.189] dB[0.098,0.181] g[6.628,6.367]
>2305, dA[0.039,0.037] dB[0.282,0.183] g[7.212,8.439]
>2306, dA[0.547,0.047] dB[0.037,0.032] g[4.063,4.909]
>2307, dA[0.034,0.174] dB[0.111,0.178] g[4.095,4.267]
>2308, dA[0.112,0.275] dB[0.236,0.081] g[4.698,4.349]
>2309, dA[0.013,0.021] dB[0.043,0.284] g[6.603,7.665]
>2310, dA[0.036,0.026] dB[0.169,0.011] g[4.420,5.261]
>2311, dA[0.062,0.022] dB[0.047,0.050] g[5.368,6.165]
>2312, dA[0.044,0.017] dB[0.011,0.039] g[5.423,5.073]
>2313, dA[0.273,0.095] dB[0.382,0.127] g[5.232,5.587]
>2314, dA[0.059,0.041] dB[0.018,0.110] g[5.067,4.985]
>2315, dA[0.089,0.238] dB[0.168,0.058] g[5.404,4.769]
>2316, dA[0.182,0.044] dB[0.061,0.136] g[5.671,5.607]
>2317, dA[0.373,0.030] dB[0.343,0.064] g[4.639,4.887]
>2318, dA[0.027,0.059] dB[0.166,0.235] g[4.354,4.253]
>2319, dA[0.108,0.434] dB[0.